# 텍스트 분할(Text Splitter)
LangChain의 텍스트 분할(Text Splitter) 은 긴 문서(예: PDF, 웹페이지, 텍스트 파일 등)를 벡터 임베딩(Vector Embedding) 단계 전에 <br>
적절한 크기의 조각(Chunk) 으로 나누기 위한 핵심 도구이다. <br>
RAG(Retrieval-Augmented Generation) 파이프라인의 성능은 이 “문서 분할 전략”에 크게 좌우된다. <br>

In [1]:
import os
from dotenv import load_dotenv

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

True

### [1] CharacterTextSplitter
: CharacterTextSplitter 는 텍스트를 문자(Character) 단위로 단순하게 자르는 기본 분할기(Text Splitter) 이다. <br>
이 클래스는 문장을 “특정 구분자(separator)” 기준으로 쪼개고, 각 조각(chunk)을 지정된 크기(chunk_size)로 묶는 방식으로 동작한다.

In [2]:
from langchain_text_splitters import CharacterTextSplitter

text = """
LangChain은 LLM 애플리케이션 개발을 위한 프레임워크입니다.
문서 로드, 텍스트 분할, 임베딩, 검색, 체인 등의 기능을 제공합니다.
CharacterTextSplitter 는 텍스트를 문자(Character) 단위로 단순하게 자르는 기본 분할기(Text Splitter) 이다.
이 클래스는 문장을 “특정 구분자(separator)” 기준으로 쪼개고, 각 조각(chunk)을 지정된 크기(chunk_size)로 묶는 방식으로 동작한다
"""

# Character 단위 분할기 생성
splitter = CharacterTextSplitter(
    separator="\n",      # 줄바꿈 기준으로 분할
    chunk_size=40,       # 각 청크 최대 40자    --> 실제는 81
    chunk_overlap=10     # 앞뒤 청크 10자 겹치기
)

# 분할 실행
chunks = splitter.split_text(text)

for i, chunk in enumerate(chunks):
     print(f"청크 {i+1}:", chunk)

# Created a chunk of size 81, which is longer than the specified 40
# LangChain의 CharacterTextSplitter 동작 방식의 핵심 특징 :
# “chunk_size는 엄격한 자르기 기준이 아니라 목표 크기(target)”이기 때문이다.
# 만약 하나의 구분자 사이 텍스트가 이미 chunk_size보다 길면,
# LangChain은 그 문장을 자르지 않고 그대로 하나의 청크로 유지한다

Created a chunk of size 81, which is longer than the specified 40


청크 1: LangChain은 LLM 애플리케이션 개발을 위한 프레임워크입니다.
청크 2: 문서 로드, 텍스트 분할, 임베딩, 검색, 체인 등의 기능을 제공합니다.
청크 3: CharacterTextSplitter 는 텍스트를 문자(Character) 단위로 단순하게 자르는 기본 분할기(Text Splitter) 이다.
청크 4: 이 클래스는 문장을 “특정 구분자(separator)” 기준으로 쪼개고, 각 조각(chunk)을 지정된 크기(chunk_size)로 묶는 방식으로 동작한다


In [3]:
from langchain_text_splitters import CharacterTextSplitter

text = """
LangChain은 LLM 애플리케이션 개발을 의 기능을 제공합니다.
CharacterTextSplitter 는 텍스트를 문자(Character) 단위위한 프레임워크입니다.
문서 로드, 텍스트 분할, 임베딩, 검색, 체인 등로 단순하게 자르는 기본 분할기(Text Splitter) 이다.
이 클래스는 문장을 “특정 구분자(separator)” 기준으로 쪼개고, 각 조각(chunk)을 지정된 크기(chunk_size)로 묶는 방식으로 동작한다
"""

# Character 단위 분할기 생성
splitter = CharacterTextSplitter(
    separator="",        # 문서를 개별 문자를 단위로 나누기 
    chunk_size=40,       # 각 청크 최대 40자   --> 40개로 나누어짐 
    chunk_overlap=10     # 앞뒤 청크 10자 겹치기
)

# 분할 실행
chunks = splitter.split_text(text)

for i, chunk in enumerate(chunks):
     print(f"청크 {i+1}:", chunk)    

청크 1: LangChain은 LLM 애플리케이션 개발을 위한 프레임워크입니다.
청크 2: 프레임워크입니다.
문서 로드, 텍스트 분할, 임베딩, 검색, 체인 등의
청크 3: 검색, 체인 등의 기능을 제공합니다.
CharacterTextSplitt
청크 4: TextSplitter 는 텍스트를 문자(Character) 단위로 단순
청크 5: er) 단위로 단순하게 자르는 기본 분할기(Text Splitter) 이
청크 6: plitter) 이다.
이 클래스는 문장을 “특정 구분자(separato
청크 7: 자(separator)” 기준으로 쪼개고, 각 조각(chunk)을 지정된
청크 8: hunk)을 지정된 크기(chunk_size)로 묶는 방식으로 동작한다


### [2] RecursiveCharacterTextSplitter
: RecursiveCharacterTextSplitter 는 LangChain에서 가장 지능적인 텍스트 분할기(Text Splitter) 이다 <br>
긴 문서를 문맥 손실 없이, LLM 입력 제한(token limit) 에 맞게 잘게 나누기 위해 설계된 클래스이다. <br>
가장 많이 쓰이고, RAG 파이프라인의 핵심. <br>

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text = """
LangChain은 LLM 애플리케이션 개발을 위한 프레임워크입니다.
문서 로드, 텍스트 분할, 임베딩, 검색, 체인 등의 기능을 제공합니다.
LangChain의 핵심 구성 요소는 체인, 메모리, 에이전트입니다.
RecursiveCharacterTextSplitter 는 LangChain에서 가장 지능적인 텍스트 분할기(Text Splitter) 이다
긴 문서를 문맥 손실 없이, LLM 입력 제한(token limit) 에 맞게 잘게 나누기 위해 설계된 클래스이다.
가장 많이 쓰이고, RAG 파이프라인의 핵심.
"""

splitter = RecursiveCharacterTextSplitter(
    chunk_size=40,          # 청크 최대 40자
    chunk_overlap=10,        # 앞뒤로 10자씩 겹침
    
    # 문자열 길이를 계산하는 함수를 지정합니다.
    length_function=len,
    
    # 구분자로 정규식을 사용할지 여부를 설정합니다.
    is_separator_regex=False,    
)

# 분할 실행
chunks = splitter.split_text(text)

for i, chunk in enumerate(chunks):
     print(f"청크 {i+1}:", chunk)  

청크 1: LangChain은 LLM 애플리케이션 개발을 위한 프레임워크입니다.
청크 2: 문서 로드, 텍스트 분할, 임베딩, 검색, 체인 등의 기능을
청크 3: 체인 등의 기능을 제공합니다.
청크 4: LangChain의 핵심 구성 요소는 체인, 메모리, 에이전트입니다.
청크 5: RecursiveCharacterTextSplitter 는
청크 6: 는 LangChain에서 가장 지능적인 텍스트 분할기(Text
청크 7: 분할기(Text Splitter) 이다
청크 8: 긴 문서를 문맥 손실 없이, LLM 입력 제한(token limit)
청크 9: limit) 에 맞게 잘게 나누기 위해 설계된 클래스이다.
청크 10: 가장 많이 쓰이고, RAG 파이프라인의 핵심.


### [3] TokenTextSplitter
: TokenTextSplitter는 텍스트를 문자 수가 아니라 토큰 단위로 나누는 클래스이다. <br>
모델의 토큰 한도(예: 8192 tokens)를 초과하지 않게 문서를 나누고, 모델의 비용/속도/성능 최적화에 유용하다.

In [8]:
from langchain_text_splitters import TokenTextSplitter

text = """
LangChain은 LLM 애플리케이션 개발을 단순화하는 프레임워크입니다.
문서 로딩, 텍스트 분할, 임베딩, 검색, 체인 구성 등의 기능을 제공합니다.
TokenTextSplitter는 텍스트를 문자 수가 아니라 토큰 단위로 나누는 클래스이다.
모델의 토큰 한도를 초과하지 않게 문서를 나누고, 모델의 비용/속도/성능 최적화에 유용하다.
"""

splitter = TokenTextSplitter(
    chunk_size=80,       # 최대 80토큰
    chunk_overlap=10     # 앞뒤 10토큰 겹치기
)

chunks = splitter.split_text(text)

for i, chunk in enumerate(chunks):
    print(f"청크 {i+1}: {chunk}\n")

# 출력된 � 기호는 청크를 자르는 과정에서 한글이 두 바이트 
# 이상으로 인코딩된 상태에서 중간이 잘려버린 현상 때문이다.
# 해결방법은 -> RecursiveCharacterTextSplitter 사용

청크 1: 
LangChain은 LLM 애플리케이션 개발을 단순화하는 프레임워크입니다.
문서 �

청크 2: �.
문서 로딩, 텍스트 분할, 임베딩, 검색, 체인 구성 등의 기능을 제�

청크 3: �능을 제공합니다.
TokenTextSplitter는 텍스트를 문자 수가 아니라 토큰 단위로 나�

청크 4: 위로 나누는 클래스이다.
모델의 토큰 한도를 초과하지 않게 문서�

청크 5: �� 문서를 나누고, 모델의 비용/속도/성능 최적화에 유용하다.


